In [16]:
from PyQt5 import QtGui, QtWidgets
from PyQt5.QtWidgets import QFileDialog
from PyQt5.QtWidgets import QApplication
import csv
from collections import namedtuple
import sys
import cv2
import numpy as np

def convert_QI_to_cv(incomingImage):
        '''  Converts a QImage into an opencv MAT format  '''

        incomingImage = incomingImage.convertToFormat(4)

        width = incomingImage.width()
        height = incomingImage.height()

        ptr = incomingImage.bits()
        ptr.setsize(incomingImage.byteCount())
        arr = np.array(ptr).reshape(height, width, 4)  #  Copies the data
        return arr

def convert_cv_to_qt(cv_img):
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)
        return convert_to_Qt_format

def draw_bb_id(image, x, y , w, h, id, color=(0, 0, 255), thickness=2):
        img_bb = cv2.rectangle(image, (x, y), (x+w, y+h), color, thickness)
        img_bb = cv2.rectangle(img_bb, (x, y - 25), (x + 100, y), color, -1)
        img_bb_id = cv2.putText(img_bb, f'id = {id}', (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.7, (255, 255, 255), thickness - 1, cv2.LINE_AA)
        return img_bb_id

def draw_bb_on_image(image, detectionData, color=(0, 0, 255), thickness=2):
        img = convert_QI_to_cv(image)
        
        for x, y , w, h, id in detectionData:
                img = draw_bb_id(img, x, y , w, h, id, color, thickness)
        
        qimage = convert_cv_to_qt(img)
        return qimage

In [17]:
image = QtGui.QImage('test_img_1.jpg')
detectionData = [(50, 50, 200, 200, 11), (200, 200, 400, 400, 12), (400, 400, 600, 600, 13)]
image = draw_bb_on_image(image, detectionData)


In [ ]:
from PyQt5 import QtGui
from PyQt5.QtWidgets import QApplication, QMainWindow
import sys
from PyQt5.QtGui import QBrush, QPen,QPainter, QPolygon
from PyQt5.QtCore import QPoint, Qt

def draw_bb_on_image(image, x, y , w, h, id, color=(0, 0, 255), thickness=2):
    painter = QPainter()
    painter.setPen(QPen(Qt.black, 5, Qt.SolidLine))
    #painter.setBrush(QBrush(Qt.red, Qt.SolidPattern))
    painter.setBrush(QBrush(Qt.red, Qt.VerPattern))
    points_bb = QPolygon([
                QPoint(x, y),
                QPoint(x, y + h),
                QPoint(x + h, y),
                QPoint(x + h, y + h)
            ])
    points_id = QPolygon([
                QPoint(x, y - 25),
                QPoint(x, y),
                QPoint(x + 100, y - 25),
                QPoint(x + 100, y)
            ])
    painter.drawPolygon(points_bb)
    painter.drawPolygon(points_id)
    